# Evaluating Fairness of AI Models in Radiology

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://githubtocolab.com/Sulam-Group/AI-Deep-Learning-Lab-2023/blob/bbjt-nb-fairness_interpretability/sessions/ai-fairness/fairness.ipynb)

---

**Before we start**

1. Change Colab runtime to GPU,
2. Add a shortcut to the shared Google Drive folder: [https://drive.google.com/drive/folders/1p90aGBS8vIX54x9ytaW8h-vk4NHXDhpR?usp=sharing](https://drive.google.com/drive/folders/1p90aGBS8vIX54x9ytaW8h-vk4NHXDhpR?usp=sharing)

In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

import os
import sys
import torch
import pandas as pd
from torch.utils.data import DataLoader
from tqdm import tqdm

LAB_PATH = os.path.join("drive/MyDrive/RSNA2023-FAIRNESS-LAB")
sys.path.append(LAB_PATH)

!python -m pip install entmax
from utils import get_dataset, get_slice_predictor

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
model = get_slice_predictor(device)

dataset = get_dataset()
dataloader = DataLoader(dataset, batch_size=1, shuffle=False)

predictions = []
for i, data in enumerate(tqdm(dataloader)):
    series, target, labels, bio = data

    series = series.squeeze()
    labels = labels.squeeze()

    series = series.to(device)

    with torch.no_grad():
        logits = model(series)

    for logit, label in zip(logits, labels):
        predictions.append(
            {
                "logit": logit.item(),
                "label": label.item(),
                "bio": bio.item(),
            }
        )

predictions_df = pd.DataFrame(predictions)
predictions_df.to_csv(os.path.join("predictions.csv"), index=False)